# Capture the Flag (RL - Policy Gradient)

- Seung Hyun Kim
- skim449@illinois.edu

## Implementation Details

- Actor-critic
- On Policy

### Sampling
- [ ] Mini-batch to update 'average' gradient
- [ ] Experience Replay for Random Sampling
- [ ] Importance Sampling
    
### Deterministic Policy Gradient
- [ ] DDPG
- [ ] MADDPG

### Stability and Reducing Variance
- [x] Gradient clipping
- [ ] Normalized Reward/Advantage
- [ ] Target Network
- [ ] TRPO
- [x] PPO

### Multiprocessing
- [ ] Synchronous Training (A2C)
- [x] Asynchronous Training (A3C)

### Applied Training Methods:
- [ ] Self-play
- [ ] Batch Policy

## Notes

- This notebook includes:
    - Building the structure of policy driven network.
    - Training with/without render
    - Saver that save model and weights to ./model directory
    - Writer that will record some necessary datas to ./logs

- This notebook does not include:
    - Simulation with RL policy
        - The simulation can be done using policy_RL.py
    - cap_test.py is changed appropriately.
    
## References :
- https://github.com/awjuliani/DeepRL-Agents/blob/master/Vanilla-Policy.ipynb (source)
- https://www.youtube.com/watch?v=PDbXPBwOavc
- https://github.com/lilianweng/deep-reinforcement-learning-gym/blob/master/playground/policies/actor_critic.py (source)
- https://github.com/spro/practical-pytorch/blob/master/reinforce-gridworld/reinforce-gridworld.ipynb

## TODO:

- Research on '_bootstrap_' instead of end-reward
- Add global step
- Think about adding discont to advantage
- Normalize reward?
- Record method in network

In [1]:
!rm -rf logs/A3C_lstm_v4_limited_7/ model/A3C_lstm_v4_limited_7

In [2]:
TRAIN_NAME='A3C_lstm_v4_limited_7'
LOG_PATH='./logs/'+TRAIN_NAME
MODEL_PATH='./model/' + TRAIN_NAME

In [3]:
import os
import configparser
#from tqdm import tqdm

import signal
import threading
import multiprocessing

import tensorflow as tf
import tensorflow.contrib.slim as slim
import tensorflow.contrib.layers as layers
from tensorflow.python.client import device_lib
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.signal

import time
import gym
import gym_cap
import gym_cap.envs.const as CONST
import numpy as np
import random
import math

# the modules that you can use to generate the policy. 
import policy.random
import policy.roomba
import policy.policy_RL
import policy.zeros

# Data Processing Module
from utility.dataModule import one_hot_encoder_v2 as one_hot_encoder
from utility.utils import MovingAverage as MA
from utility.utils import discount_rewards, normalize
from utility.buffer import Trajectory, Trajectory_buffer

from network.ActorCritic_lstm import ActorCritic as Network

%load_ext autoreload
%autoreload 2

## Hyperparameters

In [4]:
# Importing global configuration
config = configparser.ConfigParser()
config.read('config.ini')

## Environment
action_space = config.getint('DEFAULT','ACTION_SPACE')
n_agent = 1# config.getint('DEFAULT','NUM_AGENT')
map_size = 5#config.getint('DEFAULT','MAP_SIZE')
vision_range = config.getint('DEFAULT','VISION_RANGE')

## Training
total_episodes = config.getint('TRAINING','TOTAL_EPISODES')
max_ep = config.getint('TRAINING','MAX_STEP')
critic_beta = config.getfloat('TRAINING', 'CRITIC_BETA')
entropy_beta = config.getfloat('TRAINING', 'ENTROPY_BETA')
gamma = config.getfloat('TRAINING', 'DISCOUNT_RATE')

decay_lr = config.getboolean('TRAINING','DECAYING_LR')
lr_a = 1e-3 # config.getfloat('TRAINING','LR_ACTOR')
lr_c = 2e-3 # config.getfloat('TRAINING','LR_CRITIC')

## Save/Summary
save_network_frequency = config.getint('TRAINING','SAVE_NETWORK_FREQ')
save_stat_frequency = config.getint('TRAINING','SAVE_STATISTICS_FREQ')
moving_average_step = config.getint('TRAINING','MOVING_AVERAGE_SIZE')

## GPU
gpu_capacity = config.getfloat('GPU_CONFIG','GPU_CAPACITY')
gpu_allowgrow = config.getboolean('GPU_CONFIG', 'GPU_ALLOWGROW')

In [5]:
# Local configuration parameters
po_transition = 100000 # Partial observable
serial_length = 8

# Env Settings
n_channel = 11
vision_dx, vision_dy = 2*vision_range+1, 2*vision_range+1
in_size = [None,vision_dx,vision_dy,n_channel]
nenv = 8 #(int) (multiprocessing.cpu_count())

# Asynch Settings
global_scope = 'global'

## Environment Setting

In [6]:
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)
    
#Create a directory to save episode playback gifs to
if not os.path.exists(LOG_PATH):
    os.makedirs(LOG_PATH)

In [7]:
global_rewards = MA(moving_average_step)
global_ep_rewards = MA(moving_average_step)
global_length = MA(moving_average_step)
global_succeed = MA(moving_average_step)
global_episodes = 0

# Launch the session
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_capacity,
                            allow_growth=gpu_allowgrow)t

sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
progbar = tf.keras.utils.Progbar(total_episodes,interval=1)

## Environment Unit

In [8]:
class Environment(threading.Thread):
    def __init__(self, name, global_network, sess, global_step, coord):
        super(Environment, self).__init__()
        # Initialize Environment worker
        self.env = gym.make("cap-v0").unwrapped
        self.name = name
        self.global_network = global_network
        self.sess = sess
        self.global_step = global_step
        self.coord = coord
        
        self.env.reset(map_size=map_size, policy_red=policy.zeros.PolicyGen(self.env.get_map,
                                                      self.env.get_team_red))
        
        # Create AC Network for Worker
        self.local_network = Network(in_size=in_size,
                                     action_size=action_space,
                                     lr_actor=lr_a,
                                     lr_critic=lr_c,
                                     scope=self.name,
                                     #grad_clip_norm=10.0,
                                     global_step=global_step,
                                     entropy_beta=entropy_beta,
                                     sess=sess,
                                     global_network=global_ac,
                                     separate_train=True)
        

        
    def run(self, saver, writer):
        """Override Thread.run

        Note:
            Loop to run rollout
            Include summarizing and save
        """
        self.saver = saver
        self.writer = writer
        
        global global_rewards, global_ep_rewards, global_length, global_succeed, global_episodes
        total_step = 0
        while not self.coord.should_stop() and global_episodes < total_episodes:
            ep_r, r, l, s, aloss, closs, etrpy = self.rollout(init_step=total_step)
            total_step += l
            
            global_ep_rewards.append(ep_r)
            global_rewards.append(r)
            global_length.append(l)
            global_succeed.append(s)
            
            global_episodes += 1
            progbar.update(global_episodes)
            self.sess.run(global_step_next)
            
            if global_episodes % save_stat_frequency == 0 and global_episodes != 0:
                summary = tf.Summary()
                summary.value.add(tag='Records/mean_reward', simple_value=global_rewards())
                summary.value.add(tag='Records/mean_length', simple_value=global_length())
                summary.value.add(tag='Records/mean_succeed', simple_value=global_succeed())
                summary.value.add(tag='Records/mean_episode_reward', simple_value=global_ep_rewards())
                summary.value.add(tag='summary/Entropy', simple_value=etrpy)
                summary.value.add(tag='summary/actor_loss', simple_value=aloss)
                summary.value.add(tag='summary/critic_loss', simple_value=closs)
                self.writer.add_summary(summary,global_episodes)

                self.writer.flush()
                
            if global_episodes % save_network_frequency == 0 and global_episodes != 0:
                self.saver.save(self.sess, MODEL_PATH+'/ctf_policy.ckpt', global_step=global_episodes)
                
    
    def get_action(self, states, rnn_states):
        """Run graph to get action

        Args:
            state (list): list of state for each agent
            rnn_states (list): list of rnn inputs for each agent

        Returns:
            action (list) : The action for each egent
            values (list) : The value for each action for each agent
            rnn_next (list) : List of next rnn state for each agent

        Note:
            If rnn_states=None, get action without rnn_states.
        """
        actions, values, final_states = [], [], []
        for idx, state in enumerate(states):            
            action, value, final_state = self.local_network.feed_forward(
                state=state[np.newaxis,:],
                rnn_init_state=rnn_states[idx]
                )
            actions.append(action[0])
            values.append(value[0])
            final_states.append(final_state)
            
        self.feed_dict = {self.local_network.state_input_: state[np.newaxis,:],
                          self.local_network.rnn_init_states_: self.sess.run(self.local_network.rnn_init_state_single),
                          self.local_network.seq_len_: [1]}
        return actions, values, final_states
            
    def rollout(self, init_step=0):
        global global_episodes
        total_step = init_step
        with self.sess.as_default(), self.sess.graph.as_default():
            # Initialize run
            s0 = self.env.reset()
            if po_transition < global_episodes:
                s0 = one_hot_encoder(s0, self.env.get_team_blue, vision_range)
            else:
                s0 = one_hot_encoder(self.env._env, self.env.get_team_blue, vision_range)
            s0 = s0.astype(np.float)
            # parameters
            ep_r = 0 # Episodic Reward
            prev_r = 0
            step = 0
            d = False
            
            # Trajectory Buffers
            trajs = [Trajectory(depth=6) for _ in range(n_agent)]

            # RNN Initializer
            self.rnn_states = [self.sess.run(self.local_network.rnn_init_state_single) for _ in range(n_agent)]

            # Bootstrap
            a1, v1, final_states = self.get_action(s0, self.rnn_states)
            is_alive = [ag.isAlive for ag in self.env.get_team_blue]
            print(a1)
            while step <= max_ep and not d:
                a, v0 = a1, v1
                was_alive = is_alive
                self.rnn_states = final_states

                s1, rc, d, _ = self.env.step(a)
                if po_transition < global_episodes:
                    s1 = one_hot_encoder(s1, self.env.get_team_blue, vision_range)
                else:
                    s1 = one_hot_encoder(self.env._env, self.env.get_team_blue, vision_range)
                s1 = s1.astype(np.float)
                
                is_alive = [ag.isAlive for ag in self.env.get_team_blue]
                r = rc - prev_r# - 0.01
                
                if step == max_ep and d == False:
                    r = -100
                    rc = -100
                    d = True

                r /= 100.0
                ep_r += r

                if d:
                    v1 = [0.0 for _ in range(n_agent)]
                else:
                    a1, v1, final_states = self.get_action(s1, self.rnn_states)

                # push to buffer
                for idx, agent in enumerate(self.env.get_team_blue):
                    if was_alive[idx]:
                        trajs[idx].append([s0[idx],
                                           a[idx],
                                           r,
                                           v0[idx],
                                           0,
                                           self.rnn_states[idx]
                                          ])
                        
                # Iteration
                prev_r = rc
                total_step += 1
                step += 1
                s0 = s1    
            
                #self.env.render(mode='fast')
            aloss, closs, etrpy, summary_, grad_summary = self.train(trajs, v1)
            self.writer.add_summary(summary_,global_episodes)
            self.writer.add_summary(grad_summary,global_episodes)
            self.writer.flush()
        return ep_r, rc, step, self.env.blue_win, aloss, closs, etrpy
    
    def train(self, trajectories, bootstrap):
        # replay_buffer = Trajectory_buffer(depth=6)
        alosses, closses, etrpys = [],[],[]
        for idx, traj in enumerate(trajectories):
            if len(traj) == 0:
                continue
            
            # Discount Reward
            _rew = normalize(np.array(traj[2]))
            _val = np.append(traj[3],[bootstrap[idx]])  # Bootstrap
            _td  = _rew + gamma * _val[1:]
            _adv = _rew + gamma * _val[1:] - _val[:-1]
            _adv = discount_rewards(_adv, gamma)
            traj[3] = _td.tolist()
            traj[4] = _adv.tolist()

            for batch_step in range(max(1,len(traj)//serial_length//4)):
                mdp_tuples = traj.draw_trace(seq_len=serial_length)
                states, actions, rewards, td_targets, advantages, rnn_initial_states = mdp_tuples

                aloss, closs, etrpy, grad_summary = self.local_network.feed_backward(states,
                                                                       actions.astype(int),
                                                                       td_targets,
                                                                       advantages,
                                                                       np.array(self.sess.run(self.local_network.rnn_init_state_single))[:,:1], #rnn_initial_states[0],
                                                                       np.array([8]),
                                                                       retrace_lambda=0.202
                                                                       #loss_mask=0.5)
                                                                                    )
                alosses.append(aloss)
                closses.append(closs)
                etrpys.append(etrpy)
            
        summary_ = self.sess.run(merged_summary_op, self.feed_dict)
        self.local_network.pull_global()
        
        return np.mean(alosses), np.mean(closses), np.mean(etrpys), summary_, grad_summary

## Run

In [9]:
coord = tf.train.Coordinator()
# Global Network
global_step = tf.Variable(0, trainable=False, name='global_step')
global_step_next = tf.assign_add(global_step, 1)
global_ac = Network(in_size=in_size,
                    action_size=action_space,
                    scope=global_scope,
                    sess=sess,
                    global_step=global_step,
                    separate_train=True)

# Local workers
workers = []
# loop for each workers

for idx in range(nenv):#tqdm(range(nenv), ncols=65, desc="Process Initiate"):
    name = 'W_%i' % idx
    workers.append(Environment(name, global_ac, sess, global_step=global_step, coord=coord))
saver = tf.train.Saver(max_to_keep=3)
writer = tf.summary.FileWriter(LOG_PATH, sess.graph)

    
ckpt = tf.train.get_checkpoint_state(MODEL_PATH)
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
    print("Load Model : ", ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())
    print("Initialized Variables")
    

worker_threads = []
global_episodes = sess.run(global_step)

# Summarize
variable_summary = []
for var in tf.trainable_variables(scope=global_scope):
    var_name = var.name+'_var'
    var_name = var_name.replace(':','_')
    variable_summary.append(tf.summary.histogram(var_name, var))
merged_summary_op = tf.summary.merge(variable_summary)

for worker in workers:
    job = lambda: worker.run(saver, writer)
    t = threading.Thread(target=job)
    t.start()
    worker_threads.append(t)
    time.sleep(10)
coord.join(worker_threads)


/home/namsong/anaconda3/envs/py36/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


Initialized Variables
[3]


Exception in thread Thread-13:
Traceback (most recent call last):
  File "/home/namsong/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1292, in _do_call
    return fn(*args)
  File "/home/namsong/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1277, in _run_fn
    options, feed_dict, fetch_list, target_list, run_metadata)
  File "/home/namsong/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1367, in _call_tf_sessionrun
    run_metadata)
tensorflow.python.framework.errors_impl.InvalidArgumentError: Expected len(indices) == values.shape[0], but saw: 1 vs. 8
	 [[{{node W_0/actor/rnn/TensorArrayUnstack/TensorArrayScatter/TensorArrayScatterV3}} = TensorArrayScatterV3[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](W_0/actor/rnn/TensorArray_1, W_0/actor/rnn/TensorArrayUnstack/range, W_0/actor/ExpandDims, W_0/actor/rnn/TensorArray_1:1)]]

During

[3]


Exception in thread Thread-14:
Traceback (most recent call last):
  File "/home/namsong/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1292, in _do_call
    return fn(*args)
  File "/home/namsong/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1277, in _run_fn
    options, feed_dict, fetch_list, target_list, run_metadata)
  File "/home/namsong/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1367, in _call_tf_sessionrun
    run_metadata)
tensorflow.python.framework.errors_impl.InvalidArgumentError: Expected len(indices) == values.shape[0], but saw: 1 vs. 8
	 [[{{node W_1/actor/rnn/TensorArrayUnstack/TensorArrayScatter/TensorArrayScatterV3}} = TensorArrayScatterV3[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](W_1/actor/rnn/TensorArray_1, W_1/actor/rnn/TensorArrayUnstack/range, W_1/actor/ExpandDims, W_1/actor/rnn/TensorArray_1:1)]]

During

...

Exception in thread Thread-18:
Traceback (most recent call last):
  File "/home/namsong/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1292, in _do_call
    return fn(*args)
  File "/home/namsong/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1277, in _run_fn
    options, feed_dict, fetch_list, target_list, run_metadata)
  File "/home/namsong/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1367, in _call_tf_sessionrun
    run_metadata)
tensorflow.python.framework.errors_impl.InvalidArgumentError: Expected len(indices) == values.shape[0], but saw: 1 vs. 8
	 [[{{node W_5/actor/rnn/TensorArrayUnstack/TensorArrayScatter/TensorArrayScatterV3}} = TensorArrayScatterV3[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](W_5/actor/rnn/TensorArray_1, W_5/actor/rnn/TensorArrayUnstack/range, W_5/actor/ExpandDims, W_5/actor/rnn/TensorArray_1:1)]]

During

[3]


Exception in thread Thread-19:
Traceback (most recent call last):
  File "/home/namsong/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1292, in _do_call
    return fn(*args)
  File "/home/namsong/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1277, in _run_fn
    options, feed_dict, fetch_list, target_list, run_metadata)
  File "/home/namsong/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1367, in _call_tf_sessionrun
    run_metadata)
tensorflow.python.framework.errors_impl.InvalidArgumentError: Expected len(indices) == values.shape[0], but saw: 1 vs. 8
	 [[{{node W_6/actor/rnn/TensorArrayUnstack/TensorArrayScatter/TensorArrayScatterV3}} = TensorArrayScatterV3[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](W_6/actor/rnn/TensorArray_1, W_6/actor/rnn/TensorArrayUnstack/range, W_6/actor/ExpandDims, W_6/actor/rnn/TensorArray_1:1)]]

During

[4]


Exception in thread Thread-20:
Traceback (most recent call last):
  File "/home/namsong/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1292, in _do_call
    return fn(*args)
  File "/home/namsong/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1277, in _run_fn
    options, feed_dict, fetch_list, target_list, run_metadata)
  File "/home/namsong/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1367, in _call_tf_sessionrun
    run_metadata)
tensorflow.python.framework.errors_impl.InvalidArgumentError: Expected len(indices) == values.shape[0], but saw: 1 vs. 8
	 [[{{node W_7/actor/rnn/TensorArrayUnstack/TensorArrayScatter/TensorArrayScatterV3}} = TensorArrayScatterV3[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](W_7/actor/rnn/TensorArray_1, W_7/actor/rnn/TensorArrayUnstack/range, W_7/actor/ExpandDims, W_7/actor/rnn/TensorArray_1:1)]]

During